# Arxiv Chatbot

## Overview

This project implements a chatbot system with integrated tool definitions and execution. The chatbot is designed as a simple but extensible example for understanding how conversational AI can be augmented with tools, via model context protocol (MCP) to perform tasks beyond basic dialogue.

To interact with chatbot from terminal, run `arxiv_chatbot.py` 

## Import libraries

In [14]:
import arxiv
import json
import os
from typing import List
from dotenv import load_dotenv
import anthropic

## Tool Functions

In [15]:
# File name where paper metadata will be stored
PAPER_DIR = "papers"

The first tool searches for relevant arXiv papers based on a topic and stores the papers' info in a JSON file (title, authors, summary, paper url and the publication date). The JSON files are organized by topics in the papers directory. The tool does not download the papers.

In [16]:
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    Search for papers on arXiv based on a topic and store their information.

    Args:
        topic (str): The topic/keyword to search for.
        max_results (int, optional): Maximum number of results to retrieve. 
            Defaults to 5.

    Returns:
        List[str]: List of arXiv paper IDs found in the search.

    Side Effects:
        - Creates a directory under PAPER_DIR named after the topic.
        - Updates (or creates) a `papers_info.json` file with metadata of retrieved papers.
        - Prints the path where results are saved.
    """
    # Initialize arXiv API client
    client = arxiv.Client()

    # Define search query
    search = arxiv.Search(
        query=topic,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance,
    )

    # Execute the search
    papers = client.results(search)

    # Create a subdirectory for the topic (e.g., "machine_learning/")
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)

    # Path to the JSON file storing paper metadata
    file_path = os.path.join(path, "papers_info.json")

    # Load existing metadata (if any); start fresh otherwise
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # Collect paper metadata and update dictionary
    paper_ids = []
    for paper in papers:
        paper_id = paper.get_short_id()
        paper_ids.append(paper_id)

        papers_info[paper_id] = {
            "title": paper.title,
            "authors": [author.name for author in paper.authors],
            "summary": paper.summary,
            "pdf_url": paper.pdf_url,
            "published": str(paper.published.date()),
        }

    # Save updated metadata to JSON file
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)

    print(f"Results are saved in: {file_path}")

    return paper_ids

In [17]:
search_papers("mcp transport")

Results are saved in: papers\mcp_transport\papers_info.json


['2504.08999v1',
 '2508.13220v1',
 '2508.19239v1',
 '2409.10254v2',
 '2004.04606v1']

In [18]:
def extract_info(paper_id: str) -> str:
    """
    Retrieve stored metadata for a specific paper by ID.

    This function searches through all topic subdirectories in PAPER_DIR, 
    looking for a `papers_info.json` file that contains information about 
    the requested paper. If found, the paper's metadata is returned as a 
    formatted JSON string.

    Args:
        paper_id (str): The arXiv short ID of the paper to look up 
            (e.g., "2501.01234").

    Returns:
        str: JSON-formatted string with the paper's metadata if found. 
             Otherwise, a human-readable error message.

    Side Effects:
        - Prints error messages if a `papers_info.json` file cannot be read.

    Notes:
        - Paper metadata must have been previously saved by `search_papers`.
        - Metadata includes title, authors, summary, pdf_url, and publication date.
    """
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)

        # Only process directories (topic folders)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")

            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)

                        # If paper is found, return its metadata as formatted JSON
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)

                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue

    return f"There's no saved information related to paper {paper_id}."


In [19]:
# sanity check
extract_info('2504.08999v1')

'{\n  "title": "MCP Bridge: A Lightweight, LLM-Agnostic RESTful Proxy for Model Context Protocol Servers",\n  "authors": [\n    "Arash Ahmadi",\n    "Sarah Sharif",\n    "Yaser M. Banad"\n  ],\n  "summary": "Large Language Models (LLMs) are increasingly augmented with external tools\\nthrough standardized interfaces like the Model Context Protocol (MCP). However,\\ncurrent MCP implementations face critical limitations: they typically require\\nlocal process execution through STDIO transports, making them impractical for\\nresource-constrained environments like mobile devices, web browsers, and edge\\ncomputing. We present MCP Bridge, a lightweight RESTful proxy that connects to\\nmultiple MCP servers and exposes their capabilities through a unified API.\\nUnlike existing solutions, MCP Bridge is fully LLM-agnostic, supporting any\\nbackend regardless of vendor. The system implements a risk-based execution\\nmodel with three security levels standard execution, confirmation workflow, and

## Tool Schema

The `tools` list defines metadata for each function you want the LLM to call (`search_papers` and `extract_info`). Each entry describes:

1. What the tool does (`name` + `description`).

2. What inputs it requires (`input_schema`).

3. Which inputs are mandatory (`required`).

This schema acts like an API contract between the LLM and the Python functions, so the model knows how to call each tool correctly

In [20]:
tools = [
    {
        "name": "search_papers",
        "description": "Search for papers on arXiv based on a topic and store their information.",
        "input_schema": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "The topic to search for"
                }, 
                "max_results": {
                    "type": "integer",
                    "description": "Maximum number of results to retrieve",
                    "default": 5
                }
            },
            "required": ["topic"]
        }
    },
    {
        "name": "extract_info",
        "description": "Search for information about a specific paper across all topic directories.",
        "input_schema": {
            "type": "object",
            "properties": {
                "paper_id": {
                    "type": "string",
                    "description": "The ID of the paper to look for"
                }
            },
            "required": ["paper_id"]
        }
    }
]

## Tool Mapping

In [21]:
mapping_tool_function = {
    "search_papers": search_papers,
    "extract_info": extract_info
}

In [22]:
def execute_tool(tool_name: str, tool_args: dict) -> str:
    """
    Execute a registered tool function by name with provided arguments.

    This function serves as a generic dispatcher for invoking tools defined 
    in `mapping_tool_function`. It ensures results are normalized into 
    human-readable strings for consistent handling by the chatbot or LLM.

    Args:
        tool_name (str): The name of the tool to execute. Must be a key in 
            `mapping_tool_function`.
        tool_args (dict): A dictionary of arguments to pass to the tool. 
            Arguments are unpacked into keyword arguments.

    Returns:
        str: A string representation of the tool's result, normalized as follows:
            - None → "The operation completed but didn't return any results."
            - List → Comma-separated string of items
            - Dict → JSON-formatted string
            - Any other type → Converted using str()

    Raises:
        KeyError: If the provided `tool_name` is not found in `mapping_tool_function`.
    """
    result = mapping_tool_function[tool_name](**tool_args)

    if result is None:
        return "The operation completed but didn't return any results."

    if isinstance(result, list):
        return ", ".join(result)

    if isinstance(result, dict):
        return json.dumps(result, indent=2)

    return str(result)

## Chatbot Code


The chabot handles queries on a case by case basis. No memory is persistent across queries. 

In [23]:
load_dotenv() 
API_KEY = os.getenv("ANTHROPIC_API_KEY")
client = anthropic.Anthropic(api_key=API_KEY)

### Query Processing

In [24]:
def process_query(query: str) -> None:
    """
    Process a user query by interacting with the LLM and handling tool calls.

    This function sends the user's query to the LLM, monitors the response, 
    and executes tool calls when requested. It runs in a loop until the 
    assistant provides a final textual response.

    Args:
        query (str): The natural language query provided by the user.

    Behavior:
        - Sends the query and conversation history to the LLM.
        - Handles assistant text responses (prints them to stdout).
        - Detects tool call requests, executes the tool, and passes results 
          back into the conversation for the model to use.
        - Continues looping until the conversation naturally concludes.

    Notes:
        - Tools must be defined in the global `tools` schema.
        - Tool implementations must be mapped in `mapping_tool_function`.
        - The function prints outputs directly (does not return them).

    """
    messages = [{'role': 'user', 'content': query}]

    # Initial request to the model
    response = client.messages.create(
        max_tokens=2024,
        model='claude-3-7-sonnet-20250219',
        tools=tools,
        messages=messages
    )

    process_query = True
    while process_query:
        assistant_content = []

        for content in response.content:

            # Case A: Assistant responds with plain text
            if content.type == 'text':
                print(content.text)
                assistant_content.append(content)

                # End loop if this is the only content
                if len(response.content) == 1:
                    process_query = False

            # Case B: Assistant requests a tool
            elif content.type == 'tool_use':
                assistant_content.append(content)
                messages.append({'role': 'assistant', 'content': assistant_content})

                # Extract tool call details
                tool_id = content.id
                tool_args = content.input
                tool_name = content.name
                print(f"Calling tool {tool_name} with args {tool_args}")

                # Execute tool and feed result back into conversation
                result = execute_tool(tool_name, tool_args)
                messages.append({
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_id,
                            "content": result
                        }
                    ]
                })

                # Continue conversation with tool results
                response = client.messages.create(
                    max_tokens=2024,
                    model='claude-3-7-sonnet-20250219',
                    tools=tools,
                    messages=messages
                )

                # If the model follows up only with text, end loop
                if len(response.content) == 1 and response.content[0].type == "text":
                    print(response.content[0].text)
                    process_query = False


### Chat Loop

In [25]:
def chat_loop() -> None:
    """
    Start an interactive command-line chat session with the chatbot.

    This function runs a continuous loop where the user can type queries, 
    which are processed by the chatbot via `process_query`. The loop 
    terminates when the user types 'quit'.

    Behavior:
        - Prompts the user for input.
        - Passes queries to `process_query` for handling.
        - Prints chatbot responses to the console.
        - Handles errors gracefully without terminating the session.

    Returns:
        None
    """
    print("Type your queries or 'quit' to exit.")

    while True:
        try:
            query = input("\nQuery: ").strip()

            # Exit condition
            if query.lower() == 'quit':
                break

            # Process query through chatbot pipeline
            process_query(query)
            print("\n")

        except Exception as e:
            print(f"\nError: {str(e)}")


### Interact with Chatbot

In [27]:
# conversation
chat_loop()

Type your queries or 'quit' to exit.
Hello! I'm here to help you search for and extract information about academic papers on arXiv. You can ask me to:

1. Search for papers on a specific topic
2. Extract detailed information about a specific paper using its arXiv ID

What would you like to know about? You can ask something like "Find papers about quantum computing" or "Get information on paper 2104.12345".


I'll search for papers on "MCP Transport" for you. Let me use the search tool to find relevant papers on this topic.
Calling tool search_papers with args {'topic': 'MCP Transport'}
Results are saved in: papers\mcp_transport\papers_info.json
I've found 5 papers related to "MCP Transport". Let me retrieve more detailed information about each of these papers to provide you with a better understanding of their content.
Calling tool extract_info with args {'paper_id': '2504.08999v1'}
Calling tool extract_info with args {'paper_id': '2508.13220v1'}
Calling tool extract_info with args {'p